<a href="https://colab.research.google.com/github/RayBasilio123/ic2020/blob/main/hyperparametros_Grid_Arvore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score
from datetime import datetime
from matplotlib import pyplot

from math import nan
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV


In [ ]:
def get_x2(df,lista,lags,Eto,lags_eto):
  lags=lags
  lista_aux=[]
  lista_aux2=[]
  max_lag=0;
  data = pd.DataFrame()
  eta_nois = pd.DataFrame()
  eta_nois['Data']=df['Data']
  for coluna in lista:
    data[coluna] = df[coluna]       

  for i in range(len(lags)):
    for j in range(len(lags[i])):
      lista_aux = data.iloc[:,i].tolist()
      for displacement in range((lags[i][j])):
        if max_lag<(lags[i][j]):
          max_lag=(lags[i][j])
        del lista_aux[len(lista_aux)-1]
        lista_aux.insert(0,nan)
      eta_nois[((data.iloc[:,i]).name)+("_t-")+str(lags[i][j])]=(lista_aux)  
  
  for i in range(len(lags_eto)):
    lista_aux2=df[Eto].iloc[:,0].tolist()
    for displacement in range((lags_eto[i])):
      if max_lag<(lags_eto[i]):
        max_lag=(lags_eto[i])
      del lista_aux2[len(lista_aux2)-1]
      lista_aux2.insert(0,nan)
    eta_nois[((df[Eto].iloc[:,0]).name)+("_t-")+str(lags_eto[i])]=lista_aux2        
  return eta_nois,max_lag,lista,lags,Eto,lags_eto;

In [ ]:
def get_x30(df,lista, Eto):
  ix = []
  idx =  [i for i in np.arange(1, 31)]
  for i in range(len(lista)):
    ix.append(idx)

  resultado = get_x2(df, lista, ix, Eto, idx)
  return resultado

In [ ]:
def column_Filter(df,dias):
  list_column = []
  # pegando colunas do data frame
  for column in (df.columns):
    if column != 'Data':
      list_column.append(column)
  # prosseguindo com a filtração
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
 
  # filtra e  remove colunas 
  for i in range(len(list_column)):
    j=list_column[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  # print("Lista_Filtrada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

  # print("lista",lista)
  # print("Eto",Eto)
 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)

  # print("lags",lags)
           
  # print("Eto",Eto)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
  # print("lags_eto",lags_eto)
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3









In [ ]:
def resource_ranking(df,lista_filtrada3,variavel_alvo) :
  
  # resource_ranking(df_la,lista_filtrada2,"temp_min")  
  #Monta a tabela com os dados filtrados 
  
  tab=get_x2(df,lista_filtrada3[0],lista_filtrada3[1],lista_filtrada3[2],lista_filtrada3[3])
 
  # dff=tab[0].drop("Data",axis=1)
  
  # train_size = int(((len(tab[0]))-tab[1]) )
  # dff=dff.iloc[tab[1]:train_size,:]

  # array1 = dff.values

  # df= df.drop("Data",axis=1)
  # df=df.iloc[tab[1]:train_size,:]
  # array2 = df[variavel_alvo]
  selecao_treino = (tab[0]['Data'] >= '1993') & (tab[0]['Data'] <= '2011-12-31')
  
  dff=tab[0][selecao_treino].drop("Data",axis=1)

  train_size = int(((len(tab[0][selecao_treino]))-tab[1]) )
  dff=dff.iloc[tab[1]:train_size,:]
  array1 = dff.values

  df= df[selecao_treino].drop("Data",axis=1)
  df=df.iloc[tab[1]:train_size,:]
  array2 = df[variavel_alvo]

 

  X =array1[:,0:len(tab[0].columns)]
  Y = array2
  # feature extraction
  test = SelectKBest(score_func=f_regression, k=4)
  fit = test.fit(X, Y)
  # summarize scores
  set_printoptions(precision=3)
 

    
  # print("Selecao_univariada",fit.scores_)
  f=fit.scores_
  f_ord = sorted(f,reverse=True)
  # print("Selecao_univariada_ordenada",f_ord)

  ll=[]
  for y in range(len(f_ord)):
    for i in range(len(f)):
      if (f_ord[y]==f[i]):
        ll.append(i)
        # print("f[i]",f[i])
        # print("f_ord[y]",f_ord[y])
  leg_seq=[]
  for i in range(len(ll)):
    leg_seq.append(dff.columns[ll[i]])    

  # print("Colunas_selecionadas [0]- Selecao_univariada",leg_seq)
 
  model = ExtraTreesRegressor(n_estimators=10,random_state=42)
  model.fit(X,Y)
  g=model.feature_importances_
  g_ord=sorted(model.feature_importances_,reverse=True)
  # print("Importancia",g)
  # print("Importancia_ordenada",g_ord)
  jj=[]
  for y in range(len(g_ord)):
    for i in range(len(g)):
      if (g_ord[y]==g[i]):
        jj.append(i)
  leg_seq2=[]
  for i in range(len(ll)):
    leg_seq2.append(dff.columns[jj[i]])    
  # print("Colunas_selecionadas [1]- Importância do recurso",leg_seq2)
  return leg_seq,leg_seq2

In [ ]:


def list_Format(melhores_recursos,q,dias):
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
  q = len(melhores_recursos) if q==0 else q
  # filtra e  remove colunas 
  for i in range(q):
    j=melhores_recursos[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  print("lista_selecionada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

  print("lista",lista)
  print("Eto",Eto)
 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)

  print("lags",lags)
           
  # print("Eto",Eto)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
  print("lags_eto",lags_eto)
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3

In [ ]:
def train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo,data_Itest= '2012-01-01',data_Ftest= '2012-12-31',data_Itreino='1993-01-01',data_Ftreino='2011-12-31'):
  tabela = get_x2(df, lista,lags,Eto,lags_eto)
  selecao_treino = (tabela[0]['Data'] >= data_Itreino) & (tabela[0]['Data'] <= data_Ftreino)
  selecao_teste = (tabela[0]['Data'] >= data_Itest) & (tabela[0]['Data'] <= data_Ftest)

  x1_train = tabela[0][selecao_treino].drop("Data", axis=1)
  x1_test = tabela[0][selecao_teste].drop("Data", axis=1)
  
  y1_train = df[variavel_Alvo][selecao_treino]
  y1_test = df[variavel_Alvo][selecao_teste]

  x1_train = x1_train[tabela[1]:]
  y1_train = y1_train[tabela[1]:]

  
  print("***************************************")
  print(x1_train.shape,"x1_train")
  #[6569 rows x 10 columns] x1_train
  print("*****************++*********************")
  print(x1_test.shape,"x1_test")
  print("******************---********************")
  print(y1_train.shape,"y1_train")
  print("****************1234*********************")
  print(y1_test.shape,"y1_test")
  print("*****************asddfg******************")
 
  
  return x1_train, x1_test,y1_train, y1_test

In [ ]:
def arvore(df,lista, lags, Eto, lags_eto,variavel_Alvo):
  
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
  # print("***************************************")
  # print(x1_train,"x1_train")
  # #[6569 rows x 10 columns] x1_train
  # print("*****************++*********************")
  # print(x1_test,"x1_test")
  # print("******************---********************")
  # print(y1_train,"y1_train")
  # print("****************1234*********************")
  # print(y1_test,"y1_test")
  # print("*****************asddfg******************")

  model = DecisionTreeRegressor(random_state=42)
  model.fit(x1_train, y1_train)
  print("------------------")
  print("Arvore")
  y1_pred = model.predict(x1_test)
  mse = mean_squared_error(y1_test, y1_pred)
  std_mse=np.std(np.sqrt((y1_pred - y1_test)**2))
  
  print("std_mse",round(std_mse,4))
  rmse = math.sqrt(mse)
  print("Erro medio absoluto----",round(mean_absolute_error(y1_test, y1_pred),4))
  # pyplot.plot(y1_test, label='Expected tree ')
  pyplot.plot(np.arange(y1_test.shape[0]),y1_test, label='Expected tree ')
  # pyplot.plot(df["Eto"], label='variavel_Alvo ')
  pyplot.plot(y1_pred, label='Predicted tree')
  pyplot.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",
                mode="expand", borderaxespad=0, ncol=3)
  pyplot.show()
  return lista,lags,Eto,lags_eto,round(rmse,4)

def arvores(df,arvore_parametros,variavel_Alvo):
  lista_colunas=["lista","lista_lags","Eto","lags_eto","rmse"]
  tb = pd.DataFrame(columns=lista_colunas)
  print("Arvores")
  for x in range(len(arvore_parametros)):
    a=arvore(df,arvore_parametros[x][0],arvore_parametros[x][1],arvore_parametros[x][2],arvore_parametros[x][3],variavel_Alvo)
 
    tb.loc[x,'lista']=a[0]
    tb.loc[x,'lista_lags']=a[1]
    tb.loc[x,'Eto']=a[2]
    tb.loc[x,'lags_eto']=a[3]
    tb.loc[x,"rmse"]=a[4]

  print(tb)

  return tb

In [ ]:
def gridSearchF(df, lista,lags,Eto,lags_eto,variavel_Alvo):
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
   # Create the parameter grid based on the results of random search 
  param_grid = {
      'bootstrap': [True],
      'max_depth': [10,80, 90, 100, 110],
      'max_features': [2, 3],
      'min_samples_leaf': [3, 4, 5],
      'min_samples_split': [8, 10, 12],
      'n_estimators': [100, 200, 300, 1000]
  }

  # Create a base model
  rf = RandomForestRegressor(random_state = 42)

  # Instantiate the grid search model
  grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                            cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

  print(" -----------------grid_search.fit--------------")
 
  start_time=timer(None)

  grid_search.fit(x1_train, y1_train);

  timer(start_time)  
  
    
  return rf_random.best_params_

def gridSearchFs(df,parametros,variavel_Alvo):

  lista_colunas=["gridSearch"]
  tb = pd.DataFrame(columns=lista_colunas)
  
  print("gridSearchFs")
  for x in range(len(parametros)):
    a=RandomizedSearchF(df,parametros[x][0],parametros[x][1],parametros[x][2],parametros[x][3],variavel_Alvo)
 
    tb.loc[x,'Best_ramdon_hyperparameter']=[a]
  print('---------------------------------------------------------------------------------')
  

  return tb

# gridSearch Arvore

def gridSearchA(df, lista,lags,Eto,lags_eto,variavel_Alvo):
  tb = pd.DataFrame()
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
   # Create the parameter grid based on the results of random search 
  
  param_grid={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
            "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
            "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
            "max_features":["auto","log2","sqrt",None],
            "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] 
            }
 
  # Create a base model
  rf = DecisionTreeRegressor(random_state = 42)

  # Instantiate the grid search model
  grid_search=GridSearchCV(rf,param_grid=param_grid,scoring='neg_mean_squared_error',cv=3,verbose=3,n_jobs = -1,return_train_score=True)
  

  print(" -----------------grid_search.fit--------------")
  start_time=timer(None)

  grid_search.fit(x1_train, y1_train);

  timer(start_time)  

  return grid_search.best_params_

def gridSearchAs(df,parametros,variavel_Alvo):
  print('---------------------------------------------------------------------------------')
  
  for x in range(len(parametros)):
    a=gridSearchA(df,parametros[x][0],parametros[x][1],parametros[x][2],parametros[x][3],variavel_Alvo)
    
    print(x,"Best_hyperparameter_Arvore", a)
   
  print('---------------------------------------------------------------------------------')
  

  return a


In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now()
        return start_time
    elif start_time:
        thour,temp_sec=divmod((datetime.now()-start_time).total_seconds(),3600)
        tmin,tsec=divmod(temp_sec,60)

In [ ]:
tabela=pd.read_csv('https://raw.githubusercontent.com/RayBasilio123/ic2020/main/2020_Ic/Dados/data_PEto.csv')
data_Patricia=tabela.drop("Unnamed: 0",axis=1)
data_Patricia.head(2)

,Data,Tmax,Tmin,I,Tmean,UR,V,J,Eto
0,1993-01-01,31.3,15.4,11.3,22.86,64.75,0.740445,1,5.515947
1,1993-01-02,33.8,16.1,11.2,23.80,64.50,0.500000,2,5.587673


In [ ]:
atributeP= [ "Tmax","Tmean","I","UR","V","Tmin","J"]
resultadoP=get_x30(data_Patricia,atributeP,['Eto'])

In [ ]:
 for x in [1]:
   lista_Filtrada=column_Filter(resultadoP[0],x)
#   ranking_data_Patricia = resource_ranking(data_Patricia,lista_Filtrada,"Eto")
#   lista_Formatada1=list_Format(ranking_data_Patricia[0],10,0)
#   lista_Formatada2=list_Format(ranking_data_Patricia[1],10,0)
   print('\n -----------Para----' + str(x) + '----dias --------\n')
  # print("lista_Formatada_1 -------> ",lista_Formatada1)
  # print("lista_Formatada_2 -------> ",lista_Formatada2)
   print("lista_Formatada_3 -------> ",lista_Filtrada)
   print('\n ---------------------------------------------\n')


 -----------Para----1----dias --------

lista_Formatada_3 ------->  [['Tmax', 'Tmean', 'I', 'UR', 'V', 'Tmin', 'J'], [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]], ['Eto'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 

# Para  1 dia - Grid Arvore

In [ ]:

# lista_Formatada3=[['Tmax', 'Tmean', 'I', 'UR', 'V', 'Tmin', 'J'], [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]], ['Eto'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]]
lista_Formatada_1 = [['Tmean', 'Tmax'], [[1, 2, 3, 4], [1]], ['Eto'], [1, 2, 3, 4, 5]]
lista_Formatada_2 = [['Tmax', 'Tmean', 'I', 'UR', 'J'], [[1], [2], [1], [1], [22, 26]], ['Eto'], [1, 2, 3, 4]]

In [ ]:
parametrosSearchA=gridSearchAs(data_Patricia,[lista_Formatada_1,lista_Formatada_2],"Eto")

---------------------------------------------------------------------------------
***************************************
(6934, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6934,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 1578 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 4138 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 7722 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 12330 tasks      | elapsed:   57.4s
[Parallel(n_jobs=-1)]: Done 17962 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 24618 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 32298 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 41002 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 50730 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 61482 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 73258 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 86058 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 99882 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 11473

0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(6913, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6913,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 18172 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 24828 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 32508 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 50940 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 61692 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 73468 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 86268 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 100092 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1149

1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
---------------------------------------------------------------------------------
{'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'} hiperparâmetro_da_Arvore


In [ ]:
# 0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}

# 1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}

#Para  3 dias - Grid Arvore

In [ ]:
lista_Formatada_1 = [['Tmean'], [[3, 4, 5, 6, 7, 8, 9]], ['Eto'], [3, 4, 5]]
lista_Formatada_2 = [['Tmean', 'J'], [[8, 5], [26, 29, 6, 30]], ['Eto'], [3, 4, 5, 6]]

In [ ]:
parametrosSearchA=gridSearchAs(data_Patricia,[lista_Formatada_1,lista_Formatada_2],"Eto")


---------------------------------------------------------------------------------
***************************************
(6930, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6930,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done 18172 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 24828 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 32508 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 50940 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 61692 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 73468 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 86268 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 100092 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1149

0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(6909, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6909,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done 18172 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 24828 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 32508 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 50940 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 61692 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 73468 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 86268 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 100092 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1149

1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
---------------------------------------------------------------------------------
{'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'} hiperparâmetro_da_Arvore


In [ ]:
# 0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
# 1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}

# Para 7 dias Grid Arvore

In [ ]:
lista_Formatada_1 =[['Tmean', 'Tmin'], [[7, 8, 9, 10, 11, 12], [7]], ['Eto'], [7, 8, 9]]
lista_Formatada_2 =[['Tmean', 'J'], [[9, 7], [27, 30, 19, 25, 23, 8]], ['Eto'], [8, 11]]

In [ ]:
parametrosSearchA=gridSearchAs(data_Patricia,[lista_Formatada_1,lista_Formatada_2],"Eto")


---------------------------------------------------------------------------------
***************************************
(6927, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6927,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:   54.6s
[Parallel(n_jobs=-1)]: Done 18172 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 24828 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 32508 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 50940 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 61692 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 73468 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 86268 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 100092 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 1149

0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(6909, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6909,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:   57.2s
[Parallel(n_jobs=-1)]: Done 18172 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 24828 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 32508 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 50940 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 61692 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 73468 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 86268 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 100092 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 1149

1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
---------------------------------------------------------------------------------
{'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'} hiperparâmetro_da_Arvore


In [ ]:
# 0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
# 1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}

# Para 10 dias - Grid Arvore

In [ ]:
lista_Formatada_1 =[['Tmean', 'Tmin'], [[10, 11, 12, 13, 14], [10]], ['Eto'], [10, 11, 12, 13]]
lista_Formatada_2 =[['J', 'Tmean'], [[25, 17, 29, 16, 13, 30, 28], [10, 13]], ['Eto'], [10]]

In [ ]:
parametrosSearchA=gridSearchAs(data_Patricia,[lista_Formatada_1,lista_Formatada_2],"Eto")


---------------------------------------------------------------------------------
***************************************
(6925, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6925,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:   37.0s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:   56.7s
[Parallel(n_jobs=-1)]: Done 18172 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 24828 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 32508 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 50940 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 61692 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 73468 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 86268 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 100092 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 1149

0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(6909, 10) x1_train
*****************++*********************
(366, 10) x1_test
******************---********************
(6909,) y1_train
****************1234*********************
(366,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done 18172 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 24828 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 32508 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 50940 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 61692 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 73468 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 86268 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 100092 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 1149

1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
---------------------------------------------------------------------------------
{'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'} hiperparâmetro_da_Arvore


In [ ]:
# 0 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
# 1 Best_hyperparameter_Xgb {'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}